In [2]:
import numpy as np
import pandas as pd

import math
from datetime import datetime

1. Matrix Factorization을 통해서 영화 시청 이력을 기반으로 평점 예측을 해야함
2. 사용해야할 데이터셋 : movies(movieId,genre) / ratings(userId,movieId,rating,timestamp)
3. 학습 데이터: timestamp >= 1104505203 ~ timestamp <= 1230735592
4. 평가 데이터: timestamp >= 1230735600 이후

1. ratings 데이터를 트레이닝 셋과 테스트셋으로 나눔 
2. movieId-userId matrix를 생성
3. matrix factorization(SVD)으로 행렬을 분해 (유저벡터) x (영화벡터)로 나눠야함
4. 다시 복원하면 0들에 값이 채워져있음. 그게 예상하는 값인가? 그 뒤로 어케되는지 전혀 모르겠음 



## load data

In [3]:
# data load
df_ratings = pd.read_csv("../data/ratings.csv")
df_movies = pd.read_csv("../data/movies.csv") 

In [4]:
# train_test_split
df_all = df_ratings.copy()
df_train = df_all[df_all["timestamp"] >= 1104505203]
df_train = df_train[df_train["timestamp"] <= 1230735592]
df_test = df_all[df_all["timestamp"] >= 1230735600]

In [5]:
df_train.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


### bias

In [11]:
# movie preprocessing
df_movies['genres'] = df_movies['genres'].apply(lambda x : x.split("|"))
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
df_genre = df_movies.join(pd.DataFrame(mlb.fit_transform(df_movies.pop('genres')),
                         columns = mlb.classes_,
                         index = df_movies.index))

In [13]:
# row = movie, column = genre(featrue)
df_genre

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62418,209157,We (2018),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62419,209159,Window of the Soul (2001),0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
62420,209163,Bad Poems (2018),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
62421,209169,A Girl Thing (2001),1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### pivot table

In [6]:
# train set을 전처리 : row = userId column = movieId
df_train_piv = df_train.pivot("userId","movieId","rating")
df_train_piv = df_train_piv.fillna(0)
df_train_piv.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,64957,64969,64976,64983,64986,64990,64993,64997,65011,65025
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,4.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
r_ui = df_train_piv.values
r_ui.shape

(27915, 10533)

In [8]:
r_ui = r_ui + np.random.normal(1,0.1,(27915,10533))

### matrix factorization
$\hat r_{ui} = q_i^Tp_u$
* reduced = U * s @ VT 로 분해 = $q_i^Tp_u$

In [9]:
# df_train_piv를 svd decomposition
U,s,VT = np.linalg.svd(r_ui,compute_uv = True)

In [10]:
# reduced decomposition 
r = s.shape[0] - sum(np.allclose(lx,0) for lx in s)
reduced = U[:,:r]*s[:r]@VT[:r,:]
reduced

array([[1.13822307, 1.08429764, 0.91162509, ..., 1.21819465, 1.14484936,
        0.97557341],
       [4.5932748 , 0.87557434, 0.96812457, ..., 0.96302333, 0.99304581,
        0.70024033],
       [4.42940078, 1.09882449, 0.94404014, ..., 1.02481943, 1.12232117,
        0.92607284],
       ...,
       [1.01300196, 1.11660392, 1.11321395, ..., 0.99266217, 0.8347296 ,
        0.95008578],
       [5.52658407, 3.56526698, 1.29408844, ..., 0.94261705, 0.80307304,
        0.92175267],
       [5.04485409, 1.01302555, 1.26539741, ..., 0.93699234, 0.99838062,
        0.97767196]])

In [40]:
rh_ui = reduced
q_i = (U[:,:r]*s[:r]).T
p_u = VT[:r,:]
print(rh_ui.shape, q_i.shape, p_u.shape)

(27915, 10533) (10533, 27915) (10533, 10533)


In [22]:
r_ui - rh_ui

array([[-1.23243723e-15,  4.51086518e-15, -2.04546032e-15, ...,
         2.05001031e-15,  1.24851979e-15, -1.88795174e-17],
       [-1.68753900e-14,  2.42742319e-14,  2.31132420e-14, ...,
         6.36020946e-16, -1.10439544e-16,  5.49012875e-17],
       [-7.10542736e-15,  4.01292939e-15, -2.16728515e-14, ...,
         1.29228847e-15, -1.24109809e-16,  3.70255042e-17],
       ...,
       [-2.19620820e-16,  2.64763970e-18,  5.06239223e-15, ...,
         3.33361675e-16,  1.21430643e-16, -5.17503249e-17],
       [-3.01980663e-14, -2.66453526e-15, -5.86197757e-14, ...,
         1.84564244e-15,  6.65493034e-16, -9.96890016e-17],
       [-1.59872116e-14,  5.48199723e-18,  3.72006871e-15, ...,
         1.99883767e-15,  2.13155841e-18, -9.56808417e-17]])

### Learning algorithms
1. SGD(Stochastic gradient descent)
    * prediction error($e_{ui} = r_{ui} - q_i^Tp_u$)를 최소화 하는 parmeter를 gradient descent로 계산
    * $q_i \leftarrow q_i + \gamma \centerdot(e_{ui}\centerdot p_u - \lambda \centerdot q_i)$
    * $p_u \leftarrow p_u + \gamma \centerdot(e_{ui}\centerdot q_i - \lambda \centerdot p_u)$  

In [24]:
print("p_u :{} / q_i:{} /r_ui:{}".format(p_u.shape,q_i.shape,r_ui.shape))

p_u :(10326, 10533) / q_i:(10326, 27915) /r_ui:(27915, 10533)


In [12]:
np.allclose(q_i.T.dot(p_u),q_i.T@p_u)

True

In [18]:
np.vdot(q_i,q_i)

389266367.78422046

In [29]:
np.linalg.norm(q_i)

19729.834459118898

In [30]:
np.linalg.norm(p_u)

102.63040485158396

In [32]:
np.sum(p_u)

-122.39298677185161

In [41]:
gmma = 0.01 
lmda = 0.1
epoch = 10

In [42]:
for _ in range(1,epoch+1) :
    
    rh_ui = q_i.T@p_u
    
    err_ui = np.square(r_ui - rh_ui) 
    
    reg_err = np.sum(err_ui) + lmda*(np.linalg.norm(q_i)**2 + np.linalg.norm(p_u)**2)
    
    new_q_i = q_i + gmma*((p_u@err_ui.T) - (lmda * q_i))
    new_p_u = p_u + gmma*((q_i@err_ui) - (lmda * p_u))
    
    q_i,p_u = new_q_i, new_p_u
    
    print("epoch : {} / datetime : {} / loss : {}".format( _, datetime.now(), reg_err))
    if _ == 3 : break

epoch : 1 / datetime : 2020-09-08 20:29:21.930314 / loss : 38927690.07842355
epoch : 2 / datetime : 2020-09-08 20:31:14.633911 / loss : 38851429.13475191
epoch : 3 / datetime : 2020-09-08 20:33:00.986715 / loss : 146093422292.15308


In [34]:
rh_ui

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [51]:
for _ in range(1,epoch+1) :
    rh_ui = q_i.T@p_u
    
    err_ui = np.square(r_ui - rh_ui)                               
    
    p_u = p_u + gmma*((q_i@err_ui) - (lmda * p_u))
    
    print("epoch : {} / datetime : {} / loss : {}".format(_,datetime.now(), err_ui.sum()))

epoch : 1 / datetime : 2020-09-07 22:51:43.231945 / loss : 5.1958576647576585e-06
epoch : 2 / datetime : 2020-09-07 22:53:02.268791 / loss : -0.8080726455339313
epoch : 3 / datetime : 2020-09-07 22:54:13.525764 / loss : 117675.31203260839
epoch : 4 / datetime : 2020-09-07 22:55:23.679067 / loss : -17010612499.43176
epoch : 5 / datetime : 2020-09-07 22:56:25.369968 / loss : 2457107338023401.0
epoch : 6 / datetime : 2020-09-07 22:57:26.965880 / loss : -3.5489127834942616e+20
epoch : 7 / datetime : 2020-09-07 22:58:30.797872 / loss : 5.1258190563473486e+25
epoch : 8 / datetime : 2020-09-07 22:59:30.796127 / loss : -7.403394583886157e+30
epoch : 9 / datetime : 2020-09-07 23:00:31.305635 / loss : 1.0692973515749352e+36
epoch : 10 / datetime : 2020-09-07 23:01:33.655857 / loss : -1.544422366072582e+41


2. ALS(Alternating least squares)

In [44]:
(err_ui@p_u.T).shape

(27915, 10326)

In [ ]:
(np.dot(p_u.T,err_ui)).shape

In [53]:
np.square(np.array([[1,2],[3,4]]))

array([[ 1,  4],
       [ 9, 16]])